#Setup

In [1]:
pip install foolbox

In [2]:
import foolbox as fb                                                                       
import torch
import torchvision
import numpy as np
import eagerpy as ep
import matplotlib.pyplot as plt 
import matplotlib.pyplot as plt
import time
from tabulate import tabulate

## Getting the Trained Model

In [3]:
model = torchvision.models.resnet18(pretrained=True)
preprocessing = dict(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225], axis=-3)
bounds = (0, 1)
fmodel = fb.PyTorchModel(model, bounds=bounds, preprocessing=preprocessing)
fmodel = fmodel.transform_bounds((0,1))
images, labels = fb.utils.samples(fmodel, dataset='imagenet', batchsize=16)
fb.utils.accuracy(fmodel, images, labels)

/usr/local/lib/python3.7/dist-packages/foolbox/models/pytorch.py:37: UserWarning: The PyTorch model is in training mode and therefore might not be deterministic. Call the eval() method to set it in evaluation mode if this is not intended.
  "The PyTorch model is in training mode and therefore might"


0.875

In [4]:
attacks = []
attacks.append(fb.attacks.LinfDeepFoolAttack())
attacks.append(fb.attacks.FGSM())
attacks.append(fb.attacks.FGM())
attacks.append(fb.attacks.L2PGD())
attacks.append(fb.attacks.LinfPGD())
attacks.append(fb.attacks.LinfBasicIterativeAttack())
attacks.append(fb.attacks.L2ContrastReductionAttack())
attacks.append(fb.attacks.DDNAttack())
attacks.append(fb.attacks.L2BasicIterativeAttack())
attacks.append(fb.attacks.L2RepeatedAdditiveGaussianNoiseAttack())

## Running Attacks

In [5]:
images = ep.astensor(images)
labels = ep.astensor(labels)

attack_data = {'acc': [], 'times': [], 'distances': []}

for attack in attacks:

  try:
    start_time = time.perf_counter()
    raw, clipped, is_adv = attack(fmodel, images, labels, epsilons=0.03)
    elapsed = time.perf_counter() - start_time
  except:
      continue

  success = 1 - is_adv.float32().mean(axis=-1)

  attack_data['acc'].append(success.item())
  attack_data['times'].append(elapsed)
  attack_data['distances'].append(fb.distances.l0(images, clipped).numpy())

  print(f"{success} accuracy in {elapsed} seconds")


0.0 accuracy in 17.48714820999976 seconds
0.25 accuracy in 0.012426135999703547 seconds
0.8125 accuracy in 0.014178766000441101 seconds
0.8125 accuracy in 10.765897779000625 seconds
0.0 accuracy in 8.470971793999524 seconds
0.0 accuracy in 1.6960194260000208 seconds
0.875 accuracy in 0.0049311980001220945 seconds
0.8125 accuracy in 1.8230539910000516 seconds
0.75 accuracy in 1.744577591000052 seconds
0.875 accuracy in 8.528492512999946 seconds


## Comparing the Multiple Attacks

Calculate P-norms

In [6]:
def get_pnorm(p):
  pnorms = []
  for distance in attack_data['distances']:
    pnorms.append(np.linalg.norm(distance, p))
  return pnorms

attack_data['pnorm0'] = get_pnorm(0)
attack_data['pnorm1'] = get_pnorm(1)
attack_data['pnorm2'] = get_pnorm(2)
attack_data['pnormInf'] = get_pnorm(np.inf)
del attack_data['distances']

Format data

In [7]:
table = []
for i in range(0,len(attack_data['acc'])):
  row = []
  for key in attack_data:
    row.append(("%.3f" % attack_data[key][i]))
  table.append(row)


print(tabulate(table, headers=attack_data.keys()))

  acc    times    pnorm0       pnorm1    pnorm2    pnormInf
-----  -------  --------  -----------  --------  ----------
0       17.487        15  2.23778e+06    577944      150527
0.25     0.012        16  2.38854e+06    597286      150525
0.812    0.014        16  2.38773e+06    597085      150489
0.812   10.766        16  2.39566e+06    598967      150488
0        8.471        16  2.40254e+06    600646      150528
0        1.696        16  1.89345e+06    473474      124129
0.875    0.005        16  2.40845e+06    602112      150528
0.812    1.823        13  1.94529e+06    539567      150486
0.75     1.745        16  2.39106e+06    597860      150491
0.875    8.528         0  0                   0           0
